## Манифест

Как правило, документы имеют логическую структуру: название, разбиение на главы, подглавы и т. д., нумерованные и маркированные списки.
Мы занимаемся извлечением структурных элементов из сканированных документов.
Выделение такой логической структуры документа может пригодиться для автоматизированного анализа документов. Мы хотим решать эту задачу как задачу классификации, нам нужно для каждой строки текста определить, к какому типу она относится.

Выделяются следующие типы:

#### 1)Заголовок

Название главы, секции, подглавы, параграфа - если название занимает несколько строк, остальные тоже относятся к названию. Заголовок визуально выделяется от основного текста отступом и/или шрифтом и/или жирностью.

#### 2)Список

Начало нумерованного или маркированного списка - если элемент списка занимает несколько строк - остальные строки не считаем списком. К списку относятся все пронумерованные любым способом строки (если они не выделены жирностью, отступом, шрифтом и относятся к заголовкам).

#### 3)Текст

Все остальное считается текстом. В том числе, текстом считается пустая строка. Если элементы списков занимают более одной строки, то все строки, кроме первой, считаются текстом.

На вход вам будут подаваться строки документов. Вам необходимо для каждой строки документа определить её тип.
Необходимо Заголовок пометить цифрой 1, Список - 2, Текст - 3

В папке с примерами лежат изображения с правильной разметкой. Красными рамками обведены заголовки, зелёными рамками обведены элементы списков, а синими обведены текстовые блоки. 

Пример разметки.

<img width = '600px' src="examples/0040_example.jpeg">

### Уровни вложенности

#### 1)Заголовки

Как правило, в начале документа идет глобальный заголовок - будем считать его заголовком 0 уровня.

Затем могут идти подзаголовки, начинающиеся со слов Глава, Раздел, Секция, Параграф, Статья + может присутствовать нумерация. Это заголовки n-го уровня, в зависимости от нумерации. (Раздел 1 - 1 уровень, Раздел/Подраздел 1.1 - 2 уровень и т. д.). Ограничимся тремя уровнями вложенности:

0)Заголовок (может быть в несколько строк) (нулевой уровень)

1)Глава/параграф/секция/подзаголовок/раздел + нумерация

2)Подглава/подсекция/подпараграф/подраздел + нумерация

3)Подподглава и т. д. + нумерация

Также могут быть подзаголовки без нумерации и без начальных слов - строки текста, выделенные отступом и/или шрифтом и/или жирностью. Нумерация уровней определяется визуально аналогично описанной выше.

#### 2)Списки

Будем анализировать технические отчеты и статьи законов - считаем нумерованными списками следующее:

1)Вложенные списки с нумерацией 1, 1.1, 1.1.1 (и т.д.)  - это списки 1, 2, 3 (и т.д.) уровня. Выделим такие списки как отдельный класс с неограниченным уровнем вложенности. Уровень таких списков определяется отдельно.

2)Списки + уровень вложенности (ограничимся третьим уровнем вложенности)

-Строки, начинающиеся с цифр (нумерованные списки) 1. 1) 1

-Списки, в которых нумерация ведется с помощью букв а), А), а, А, (a), (A)

-Списки маркированные точка, галка, кружок, тире, ромб, квадрат, крестик и т.д.

Если один список вложен в другой или список относится к блоку текста с заголовком n-го уровня, то уровень вложенности такого списка равен n+1, то есть уровень увеличивается на единицу по сравнению с предыдущим элементом.

Заголовки и элементы списков могут занимать несколько строк, при этом каждая строка заголовка считается заголовком, а в элементах списков списком считается только первая строка, остальные строки - текст (но уровень вложенности такой же, как и у списка, к которому относится строка)

#### 3)Просто строки 

Уровень вложенности определяется блоком текста, к которому относится данная строка. Например, если строка - продолжение элемента списка, то ее уровень вложенности соответствует уровню вложенности элемента списка. Если строка располагается после подзаголовка n-го уровня, то ее уровень также равен n.

### Вопросы на будущее

Что делать с английскими буквами? (или римскими цифрами, например)

Как извлекать информацию о шрифтах, отступах и т.д.?

Насколько сложная структура может быть (насколько сложные и разнообразные документы)? 

Как оценивать вместе с номером уровня заголовка?


## Извлечение текста из pdf
https://www.severcart.ru/blog/all/tesseract_ocr_python/

In [1]:
from pdf2text import pdf2text

## Классификация заголовков

In [2]:
import re
RE_LIST = re.compile(r'\d+(\.\d+)*\D') # для отдельного типа списка
RE_HEADER = re.compile(r'Раздел|Подраздел|Глава|Параграф|Секция|Часть|Статья')

In [ ]:
def list_proc(line):
    """
    если список, возвращает тип и глубину вложенности
    """
    match = RE_LIST.search(line)
    if match:
        if match.start() == 0:
            return 'list', match.string[match.span()[0]:match.span()[1]].count('.') + 1
    return

In [3]:
class AddFeatures:
    def __init__(self):
        pass
    def fit(self):
        pass
    def transform(self, X):
        """
        returns 2 columns: 1-list, 2-header
        X - list of strings
        """
        features = []
        for line in X:
            match = RE_LIST.search(line)
            if match:
                if match.start() == 0:
                    features.append([1, 0])
                    continue
            match = RE_HEADER.search(line)
            if match:
                if match.start() == 0:
                    features.append([0, 1])
                    continue
            features.append([0, 0])
        return features
    def fit_transform(self, X, y=None):
        return self.transform(X)

Извлекаем признаки:
смотрим на первое слово в каждой непустой строчке документа -> считаем слова

In [4]:
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from string2features import string2features

ppl = make_pipeline(string2features(), 
                    FeatureUnion([('af', AddFeatures()),
                                  ('cv', CountVectorizer(token_pattern=r'(?u)\b\w+\b'))]), 
                    LogisticRegression())

Обучаем модель:
логистическая регрессия, для каждой строки - тип строки + уровень???

как работать с уровнем? строка-название + строка-уровень

In [5]:
import json
with open("file_train.json", "r") as read_file:
    doc_with_labels = json.load(read_file)

In [6]:
y = [x["type"] for x in doc_with_labels]
X = [x["content"] for x in doc_with_labels]

Обучаем и сохраняем обученную модель

In [7]:
import pickle as pkl
clf = ppl.fit(X, y)
pkl.dump(clf, open("model.pkl", "wb"))

/home/nastyboget/media/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/nastyboget/media/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Кросс-валидация

In [8]:
with open("file_with_labels.json", "r") as read_file:
    doc_test = json.load(read_file)
    y = [x["type"] for x in doc_test]
    X = [x["content"] for x in doc_test]

In [9]:
from sklearn.model_selection import cross_validate
clf = make_pipeline(string2features(), CountVectorizer(token_pattern=r'(?u)\b\w+\b'), LogisticRegression())
scores = cross_validate(clf, X, y, cv=3, scoring='f1_macro')
scores

/home/nastyboget/media/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/nastyboget/media/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/nastyboget/media/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/nastyboget/media/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/nastyboget/media/anaconda3/lib/pyt

{'fit_time': array([0.02383327, 0.02338219, 0.02320027]),
 'score_time': array([0.07023168, 0.01454997, 0.00963664]),
 'test_score': array([0.59539095, 0.52446103, 0.5210825 ])}

In [10]:
import numpy as np
score = np.mean(scores['test_score'])
score

0.546978158505072